In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

/Users/junweiluo/anaconda3/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/junweiluo/anaconda3/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/junweiluo/anaconda3/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/junweiluo/anaconda3/lib/python3.7/site-pac

In [2]:
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()
X_train_full = X_train_full / 255.0
X_test = X_test / 255.0
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

In [3]:
def split_dataset(X, y):
    y_5_or_6 = (y == 5) | (y == 6) # sandals or shirts
    y_A = y[~y_5_or_6]
    y_A[y_A > 6] -= 2 # class indices 7, 8, 9 should be moved to 5, 6, 7
    y_B = (y[y_5_or_6] == 6).astype(np.float32) # binary classification task: is it a shirt (class 6)?
    return ((X[~y_5_or_6], y_A),
            (X[y_5_or_6], y_B))

(X_train_A, y_train_A), (X_train_B, y_train_B) = split_dataset(X_train, y_train)
(X_valid_A, y_valid_A), (X_valid_B, y_valid_B) = split_dataset(X_valid, y_valid)
(X_test_A, y_test_A), (X_test_B, y_test_B) = split_dataset(X_test, y_test)
X_train_B = X_train_B[:200]
y_train_B = y_train_B[:200]

In [4]:
X_train_A.shape

(43986, 28, 28)

In [5]:
model_A = keras.models.Sequential()
model_A.add(keras.layers.Flatten(input_shape=[28, 28]))
for n_hidden in (300, 100, 50, 50, 50):
    model_A.add(keras.layers.Dense(n_hidden, activation="selu"))
model_A.add(keras.layers.Dense(8, activation="softmax"))

In [6]:
model_A.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 300)               235500    
_________________________________________________________________
dense_1 (Dense)              (None, 100)               30100     
_________________________________________________________________
dense_2 (Dense)              (None, 50)                5050      
_________________________________________________________________
dense_3 (Dense)              (None, 50)                2550      
_________________________________________________________________
dense_4 (Dense)              (None, 50)                2550      
_________________________________________________________________
dense_5 (Dense)              (None, 8)                 4

In [7]:
model_A.compile(loss="sparse_categorical_crossentropy",
                optimizer=keras.optimizers.SGD(lr=1e-3),
                metrics=["accuracy"])

In [8]:
history = model_A.fit(X_train_A, y_train_A, epochs=20,
                    validation_data=(X_valid_A, y_valid_A))

Train on 43986 samples, validate on 4014 samples
Epoch 1/20
43986/43986 [==============================] - 5s 105us/sample - loss: 0.5504 - accuracy: 0.8190 - val_loss: 0.3747 - val_accuracy: 0.8747
Epoch 2/20
43986/43986 [==============================] - 4s 94us/sample - loss: 0.3484 - accuracy: 0.8800 - val_loss: 0.3198 - val_accuracy: 0.8936
Epoch 3/20
43986/43986 [==============================] - 3s 74us/sample - loss: 0.3130 - accuracy: 0.8909 - val_loss: 0.3062 - val_accuracy: 0.8981
Epoch 4/20
43986/43986 [==============================] - 3s 73us/sample - loss: 0.2940 - accuracy: 0.8984 - val_loss: 0.2853 - val_accuracy: 0.9068
Epoch 5/20
43986/43986 [==============================] - 3s 70us/sample - loss: 0.2808 - accuracy: 0.9031 - val_loss: 0.2779 - val_accuracy: 0.9088
Epoch 6/20
43986/43986 [==============================] - 3s 76us/sample - loss: 0.2718 - accuracy: 0.9068 - val_loss: 0.2662 - val_accuracy: 0.9133
Epoch 7/20
43986/43986 [==============================] 

In [9]:
model_A.save("model_A.h5")

In [11]:
model_A = keras.models.load_model("model_A.h5")

In [12]:
model_B_on_A = keras.models.Sequential(model_A.layers[:-1])

In [13]:
model_B_on_A.add(keras.layers.Dense(1, activation='sigmoid'))